In [1]:
import utils.helper_functions as hf

import os

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter

import hyperopt
from hyperopt import hp, fmin, tpe

pd.options.mode.copy_on_write = True # avoid making unnecessary copies of DataFrames or Series

db_path = './db/ohlcv_ntickers_593_2000-08-01_to_2023-12-23.pkl'
transformed_data_path = './outputs/classifier_transformed_data.pkl'
model_path = './outputs/classifier_model.keras'

fee = 0.002
# num_tickers = 1250

use_hyperopt = True
use_saved_transformed_data = False
use_saved_model = False

start_date = '2013-01-01'
test_size = 100000
epochs = 3
hyperopt_n_iterations = 600

param_grid = {
    'buying_time': ['Open'], 'selling_time': ['Open'],
    'target_future_days': [1], 'loss_limit': [0.985],
    'sell_at_target': [False],
    'size_layer_1': [128], 'size_layer_2': [128], 'size_layer_3': [64],
    'dropout_rate': [0.075], 'balance_data': [True], 'batch_size': [128], #'dropout_rates': [i for i in list(np.arange(0, 0.3, 0.1))], 'batch_sizes': [32, 64, 128],
    'n_first_classes': [[0,2]],
    'cumulated_probs_target': [0.95],
    'thresholds': [[1.08, 1.04, 1.02, 1.005]]
}
num_combinations = hf.get_num_combinations(param_grid)

search_space = {
    'buying_time': hp.choice('buying_time', ['Open', 'Close']),
    'selling_time': hp.choice('selling_time', ['Open', 'Close']),
    'target_future_days': hp.randint('param', 1, 91),
    'loss_limit': hp.uniform('loss_limit', 0.8, 1),
    'sell_at_target': hp.choice('sell_at_target', [True, False]),
    'size_layer_1': hp.choice('size_layer_1', [128]),
    'size_layer_2': hp.choice('size_layer_2', [128]),
    'size_layer_3': hp.choice('size_layer_3', [128, 256]),
    'dropout_rate': hp.uniform('dropout_rate', 0, 0.3),
    'balance_data': hp.choice('balance_data', [True]),
    'batch_size': hp.choice('batch_size', [32, 64, 128]),
    'n_first_classes': hp.choice('n_first_classes', [[0, 0], [0, 1], [0, 2], [1, 1], [1, 2], [2, 2]]),
    'cumulated_probs_target': hp.uniform('cumulated_probs_target', 0.5, 1),
    'thresholds': hp.choice('thresholds', [[1.08, 1.04, 1.02, 1], [1.06, 1.03, 1.01], [1.05, 1.025, 1], [1.1, 1.05, 1.01]]),
}

number of combinations: 1


In [2]:
df = pd.read_pickle(db_path)
df = hf.get_rows_after_date(df, start_date)

def get_single_level_df(df, ohlcv):
    new_df = df[[ohlcv]]
    new_df = hf.remove_top_column_name(new_df)

    return new_df

def get_ohlcv_dfs(df):
    df_open = get_single_level_df(df, 'Open')
    df_high = get_single_level_df(df, 'High')
    df_low = get_single_level_df(df, 'Low')
    df_close = get_single_level_df(df, 'Close')
    df_volume = get_single_level_df(df, 'Volume')
    
    return {'df_open': df_open, 'df_high': df_high, 'df_low': df_low,
            'df_close': df_close, 'df_volume': df_volume}

num_tickers = hf.get_num_tickers(get_single_level_df(df, 'Open'))
print(f'number of tickers: {num_tickers}')


number of tickers: 593


In [3]:
def calculate_var(df, past_days, future_days):
    var = hf.calculate_variations(df, past_days, future_days)
    var_stacked = hf.stack(var, f'input_var_past_{past_days}d_future_{future_days}d')

    return var_stacked

def calculate_var_vs_past_ohlcv(df, df_past, past_days, title):
    var = df / df_past.shift(past_days)
    var_stacked = hf.stack(var, f'input_var_past_{title}_{past_days}d')

    return var_stacked

def calculate_volume_var(df_volume, past_start_day, past_end_day):
    df_volume.replace(0, np.nan, inplace=True)
    df_volume.replace([np.inf, -np.inf], np.nan, inplace=True)

    volume_var= df_volume.shift(past_end_day) / df_volume.shift(past_start_day)
    volume_var_stacked = hf.stack(volume_var, f'input_volume_var_{past_start_day}-{past_end_day}d')

    return volume_var_stacked

def calculate_market_var(df, past_days):
    market_var = hf.calculate_market_variations(df, past_days)
    market_var_stacked = hf.stack(market_var, f'input_market_var_{past_days}d')

    return market_var_stacked

def min_max_var(df, past_days):
    rolling_min = df.rolling(window=past_days + 1, min_periods=1).min()
    min_var = df / rolling_min
    min_var_stacked = hf.stack(min_var, f'input_min_var_past_{past_days}d')

    rolling_max = df.rolling(window=past_days + 1, min_periods=1).max()
    max_var = df / rolling_max
    max_var_stacked = hf.stack(max_var, f'input_max_var_past_{past_days}d')

    return min_var_stacked, max_var_stacked

def get_future_end_var(df_buy, df_sell, future_days):
    df_future_end = df_sell.shift(-future_days)
    future_end_var =  df_future_end / df_buy
    future_end_var_stacked = hf.stack(future_end_var, f'output_future_end_var')
    
    return future_end_var_stacked

def get_future_max_var(df_buy, df_sell, future_days):
    future_rolling_max = hf.get_future_rolling_max(df_sell, future_days+1)
    future_max_var = future_rolling_max / df_buy
    future_max_var_stacked = hf.stack(future_max_var, f'output_future_max_var')
        
    return future_max_var_stacked

def get_future_min_var(df_buy, df_low, future_days):
    future_rolling_min = hf.get_future_rolling_min(df_low, future_days+1)
    future_min_var = future_rolling_min / df_buy
    future_min_var_stacked = hf.stack(future_min_var, f'output_future_min_var')
        
    return future_min_var_stacked

def get_future_min_var_before_max(df_buy, df_sell, df_low, future_days):
    rolling_max_positions = hf.get_future_rolling_max_position(df_sell, future_days)

    df_low = df_low.reset_index(drop=True)
    rolling_min = df_low.apply(lambda col: col.index.map(
            lambda row: hf.get_future_rolling_min_value(row, df_low.columns.get_loc(col.name), df_low, rolling_max_positions)
        ))
    rolling_min.index = df_buy.index
    
    var = rolling_min / df_buy
    var_stacked = hf.stack(var, f'output_future_min_var_before_max')

    return var_stacked

def days_since_min_max(df, past_days):
    days_since_min = hf.get_days_since_min(df, past_days)
    days_since_min_stacked = hf.stack(days_since_min, f'input_days_since_min_{past_days}d')

    days_since_max = hf.get_days_since_max(df, past_days)
    days_since_max_stacked = hf.stack(days_since_max, f'input_days_since_max_{past_days}d')

    return days_since_min_stacked, days_since_max_stacked

def get_volatility(df, past_days):
    volatility = hf.calculate_volatility(df, past_days)
    volatility_stacked = hf.stack(volatility, f'input_volatility_{past_days}d')

    return volatility_stacked

def get_market_volatility(df, past_days):
    market_average = hf.calculate_averages(df)
    volatility = hf.calculate_volatility(market_average, past_days)
    volatility_stacked = hf.stack(volatility, f'input_market_volatility_{past_days}d')

    return volatility_stacked

def get_volume_volability(df, past_days):
    volatility = hf.calculate_volatility(df, past_days)
    volatility_stacked = hf.stack(volatility, f'input_volume_volatility_{past_days}d')

    return volatility_stacked

def get_n_ups(df, past_days):
    n_ups = hf.calculate_n_ups(df, past_days)
    n_ups_stacked = hf.stack(n_ups, f'input_n_ups_{past_days}d')

    return n_ups_stacked

def get_rank(df, past_days, future_days):
    rank = hf.calculate_rank(df, past_days, future_days)
    
    if future_days == 0:
        rank_stacked = hf.stack(rank, f'input_rank_{past_days}d')
    elif past_days == 0:
        rank_stacked = hf.stack(rank, f'output_rank_{future_days}d')
    else:
        raise ValueError('Either past_days or future_days must be 0')
    
    return rank_stacked

def get_performance_vs_market(df, past_days):
    performance_vs_market = hf.calculate_performance_vs_market(df, past_days)
    performance_vs_market_stacked = hf.stack(performance_vs_market, f'input_perf_vs_market_{past_days}d')

    return performance_vs_market_stacked

def classify_var(df_var, thresholds, col_name):
    df_thresholds = hf.classify_var(df_var, thresholds)

    df_thresholds_stacked = hf.stack(df_thresholds, col_name)
    df_thresholds_stacked = df_thresholds_stacked.droplevel(level=-1)

    return df_thresholds_stacked


In [4]:
def get_inputs(df_buy, dfs_ohlcv):
    var_90 = calculate_var(df_buy, past_days=90, future_days=0)
    var_60 = calculate_var(df_buy, past_days=60, future_days=0)
    var_30 = calculate_var(df_buy, past_days=30, future_days=0)
    var_10 = calculate_var(df_buy, past_days=10, future_days=0)
    var_5 = calculate_var(df_buy, past_days=5, future_days=0)
    var_2 = calculate_var(df_buy, past_days=2, future_days=0)
    var_1 = calculate_var(df_buy, past_days=1, future_days=0)

    var_vs_close_1 = calculate_var_vs_past_ohlcv(df_buy, dfs_ohlcv['df_close'], past_days=1, title='close')
    var_vs_low_1 = calculate_var_vs_past_ohlcv(df_buy, dfs_ohlcv['df_low'], past_days=1, title='low')
    var_vs_high_1 = calculate_var_vs_past_ohlcv(df_buy, dfs_ohlcv['df_high'], past_days=1, title='high')

    volume_var_90_1 = calculate_volume_var(dfs_ohlcv['df_volume'], past_start_day=90, past_end_day=1)
    volume_var_60_1 = calculate_volume_var(dfs_ohlcv['df_volume'], past_start_day=30, past_end_day=1)
    volume_var_30_1 = calculate_volume_var(dfs_ohlcv['df_volume'], past_start_day=30, past_end_day=1)
    volume_var_10_1 = calculate_volume_var(dfs_ohlcv['df_volume'], past_start_day=10, past_end_day=1)
    volume_var_3_1 = calculate_volume_var(dfs_ohlcv['df_volume'], past_start_day=3, past_end_day=1)
    volume_var_2_1 = calculate_volume_var(dfs_ohlcv['df_volume'], past_start_day=2, past_end_day=1)
    
    # market_var_30 = calculate_market_var(df_buy, past_days=30)
    # market_var_5 = calculate_market_var(df_buy, past_days=5)
    # market_var_1 = calculate_market_var(df_buy, past_days=1)
    
    min_var_90, max_var_90 = min_max_var(df_buy, past_days=90)
    min_var_30, max_var_30 = min_max_var(df_buy, past_days=30)
    min_var_10, max_var_10 = min_max_var(df_buy, past_days=10)
    min_var_5, max_var_5 = min_max_var(df_buy, past_days=5)
    min_var_2, max_var_2 = min_max_var(df_buy, past_days=2)

    days_since_min_30, days_since_max_30 = days_since_min_max(df_buy, past_days=30)
    days_since_min_10, days_since_max_10 = days_since_min_max(df_buy, past_days=10)

    # volatility_30 = get_volatility(df_buy, past_days=30)
    # volatility_10 = get_volatility(df_buy, past_days=10)
    # volatility_2 = get_volatility(df_buy, past_days=2)

    # market_volatility_30 = get_market_volatility(df_buy, past_days=30)
    # market_volatility_10 = get_market_volatility(df_buy, past_days=10)
    # market_volatility_2 = get_market_volatility(df_buy, past_days=2)

    volume_volability_90 = get_volume_volability(dfs_ohlcv['df_volume'], past_days=90)
    volume_volability_30 = get_volume_volability(dfs_ohlcv['df_volume'], past_days=30)
    volume_volability_10 = get_volume_volability(dfs_ohlcv['df_volume'], past_days=10)
    volume_volability_2 = get_volume_volability(dfs_ohlcv['df_volume'], past_days=2)

    n_ups_90 = get_n_ups(df_buy, past_days=90)
    n_ups_30 = get_n_ups(df_buy, past_days=30)
    n_ups_5 = get_n_ups(df_buy, past_days=5)

    rank_90 = get_rank(df_buy, past_days=90, future_days=0)
    rank_30 = get_rank(df_buy, past_days=30, future_days=0)
    rank_10 = get_rank(df_buy, past_days=10, future_days=0)
    rank_5 = get_rank(df_buy, past_days=5, future_days=0)
    rank_2 = get_rank(df_buy, past_days=2, future_days=0)
    rank_1 = get_rank(df_buy, past_days=1, future_days=0)

    perf_vs_market_90 = get_performance_vs_market(df_buy, past_days=90)
    perf_vs_market_30 = get_performance_vs_market(df_buy, past_days=30)
    perf_vs_market_10 = get_performance_vs_market(df_buy, past_days=10)
    perf_vs_market_5 = get_performance_vs_market(df_buy, past_days=5)
    perf_vs_market_2 = get_performance_vs_market(df_buy, past_days=2)
    perf_vs_market_1 = get_performance_vs_market(df_buy, past_days=1)

    df_data = pd.concat([
            var_90, var_60, var_30, var_10, var_5, var_2, var_1,
            var_vs_close_1, var_vs_high_1, var_vs_low_1,
            volume_var_90_1, volume_var_60_1, volume_var_30_1, volume_var_10_1, volume_var_2_1, volume_var_3_1,
            min_var_90, min_var_30, min_var_10, min_var_5, min_var_2,
            max_var_90, max_var_30, max_var_10, max_var_5, max_var_2,
            days_since_min_30, days_since_min_10,
            days_since_max_30, days_since_max_10,
            volume_volability_90, volume_volability_30, volume_volability_10, volume_volability_2,
            n_ups_90, n_ups_30, n_ups_5,
            rank_90, rank_30, rank_10, rank_5, rank_2, rank_1,
            perf_vs_market_90, perf_vs_market_30, perf_vs_market_10, perf_vs_market_5,
            perf_vs_market_2, perf_vs_market_1
        ], axis='columns')

    df_data = df_data.dropna()

    return df_data

In [5]:
def add_future_vars(df_data, df_buy, df_sell, dfs_ohlcv, **hyperparams):
    target_future_days = hyperparams.get('target_future_days')
    sell_at_target = hyperparams.get('sell_at_target')
    
    future_end_var = get_future_end_var(df_buy, df_sell, target_future_days)
    future_max_var = get_future_max_var(df_buy, df_sell, target_future_days)
    future_min_var = get_future_min_var(df_buy, dfs_ohlcv['df_low'], target_future_days-1)

    df_data = pd.concat([df_data, future_end_var, future_max_var, future_min_var], axis='columns')
    
    if sell_at_target:
        future_min_var_before_max = get_future_min_var_before_max(df_buy, df_sell, dfs_ohlcv['df_low'], target_future_days)
        df_data = pd.concat([df_data, future_min_var_before_max], axis='columns')
    
    return df_data

def add_output_loss_min_var(df, **hyperparams):
    sell_at_target = hyperparams.get('sell_at_target')
    
    if sell_at_target:
        df['output_loss_min_var'] = df['output_future_min_var_before_max']
    else:
        df['output_loss_min_var'] = df['output_future_min_var']

    return df

def add_output_is_loss_limit_reached(df, **hyperparams):
    loss_limit = hyperparams.get('loss_limit')

    df['output_is_loss_limit_reached'] = (df['output_loss_min_var'] <= loss_limit)

    return df

def add_output_class(df_data, **hyperparams):
    sell_at_target = hyperparams.get('sell_at_target')
    thresholds = hyperparams.get('thresholds')
    last_class = len(thresholds)

    if sell_at_target:
        output_class = classify_var(df_data[['output_future_max_var']], thresholds, 'output_class')
    else:
        output_class = classify_var(df_data[['output_future_end_var']], thresholds, 'output_class')

    output_class.loc[df_data['output_is_loss_limit_reached'], 'output_class'] = last_class
    
    df_data = pd.concat([df_data, output_class], axis='columns')

    return df_data

def add_output_is_buy(df, **hyperparams):
    accepted_n_first_classes = hyperparams.get('n_first_classes')[1]
    df['output_is_buy'] = (df['output_class'] <= accepted_n_first_classes)
    
    return df

def add_output_profit(df, fee, **hyperparams):
    thresholds = hyperparams.get('thresholds')
    accepted_n_first_classes = hyperparams.get('n_first_classes')[1]
    loss_limit = hyperparams.get('loss_limit')
    sell_at_target = hyperparams.get('sell_at_target')

    accepted_var = thresholds[accepted_n_first_classes]

    condition1 = (df['output_loss_min_var'] <= loss_limit)
    condition2 = (df['output_is_buy'] & sell_at_target)

    df['output_profit'] = np.where(
        condition1 | condition2,
        np.where(
            condition1,
            loss_limit,
            accepted_var),
        df['output_future_end_var']
    )

    fee_coef = hf.get_fee_coef(fee)
    df['output_profit'] *= fee_coef
    
    return df

In [6]:
def get_df_data(hyperparams):
    df_buy = get_single_level_df(df, hyperparams['buying_time'])
    df_sell = get_single_level_df(df, hyperparams['selling_time'])
    dfs_ohlcv = get_ohlcv_dfs(df)

    if os.path.exists(transformed_data_path) and use_saved_transformed_data:
        df_data = pd.read_pickle(transformed_data_path)
        print(f'using existing {transformed_data_path}')
    else:
        print(f'need to create {transformed_data_path}')
        df_data = get_inputs(df_buy, dfs_ohlcv)
        
        df_data.to_pickle(transformed_data_path)
        print(f'saved new {transformed_data_path}')

    df_data = add_future_vars(df_data, df_buy, df_sell, dfs_ohlcv, **hyperparams)
    df_data = add_output_loss_min_var(df_data, **hyperparams)
    df_data = add_output_is_loss_limit_reached(df_data, **hyperparams)
    df_data = add_output_class(df_data, **hyperparams)
    df_data = add_output_is_buy(df_data, **hyperparams)
    df_data = add_output_profit(df_data, fee, **hyperparams)
    df_data = df_data.dropna()

    return df_data

In [7]:
def get_dfs_input_output(df_data):
    input_columns = [col for col in df_data.columns if col.startswith('input_')]
    df_input = df_data[input_columns]
    df_output = df_data[['output_class']]

    return df_input, df_output

def get_test_train_data(df_input, df_output, test_size):
    X_train = df_input[:-test_size].values
    y_train = df_output[:-test_size].values.ravel().astype(int)

    X_test = df_input.tail(test_size).values
    y_test = df_output.tail(test_size).values.ravel().astype(int)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    print(f"Number of elements in y_train: {len(y_train)}")
    print(f"Number of elements in y_test: {len(y_test)}")

    return {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}

def create_model(**kwargs):
    X_train = kwargs.get('X_train')
    X_test = kwargs.get('X_test')
    y_train = kwargs.get('y_train')
    y_test = kwargs.get('y_test')

    thresholds = kwargs.get('thresholds')
    
    size_layer_1 = kwargs.get('size_layer_1')
    size_layer_2 = kwargs.get('size_layer_2')
    size_layer_3 = kwargs.get('size_layer_3')
    dropout_rate = kwargs.get('dropout_rate')
    balance_data = kwargs.get('balance_data')
    batch_size = kwargs.get('batch_size')

    last_layers_size = len(thresholds) + 1

    model = Sequential()

    model.add(Dense(size_layer_1, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(size_layer_2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(size_layer_3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(last_layers_size, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    if (balance_data):
        counter = Counter(y_train)
        max_count = max(counter.values())
        class_weights = {cls: max_count / count for cls, count in counter.items()}
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), class_weight=class_weights)
    else:
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

    model.save(model_path)

def load_model(df_data, hyperparams):
    df_input, df_output = get_dfs_input_output(df_data)
    test_train_data = get_test_train_data(df_input, df_output, test_size)

    if os.path.exists(model_path) and use_saved_model:
        print(f'using existing {model_path}')
    else:
        print(f'need to create {model_path}')
        create_model(**{**test_train_data, **hyperparams})
    
    model = tf.keras.models.load_model(model_path)

    return test_train_data, model

In [8]:
def slice_df_test(df_data, test_size):
    return df_data.tail(test_size)

def add_predictions(df, model, X_test, **hyperparams):
    predicted_n_first_classes = hyperparams['n_first_classes'][0]
    cumulated_probs_target = hyperparams['cumulated_probs_target']

    prediction_y_test_lists = model.predict(X_test)
    prediction_y_test_array = np.array(prediction_y_test_lists)
    df['prediction_probs'] = prediction_y_test_array.tolist()

    df['prediction_cumulated_probs'] = [sum(row[:predicted_n_first_classes+1]) for row in df['prediction_probs']]
    df['prediction_is_buy'] = (df['prediction_cumulated_probs'] > cumulated_probs_target)
    df['prediction_is_correct'] = (df['output_is_buy'] == df['prediction_is_buy'])

    return df

def get_class_cumulative_percentages(y_test):
    unique_values, counts = np.unique(y_test, return_counts=True)
    percentages = counts / len(y_test)
    percentages = percentages[np.argsort(unique_values)]
    cumulative_percentages = np.cumsum(percentages)

    print(f'market cumulative % per class: {cumulative_percentages}')

    return cumulative_percentages

def get_market_rate(y_test, **hyperparams):
    accepted_n_first_classes = hyperparams['n_first_classes'][1]

    class_cumulative_percentages = get_class_cumulative_percentages(y_test)
    market_rate = class_cumulative_percentages[accepted_n_first_classes]

    return market_rate

def get_binary_classification(df):
    # tp: true positive, tn: true negative, fp: false positive, fn: false negative  
    tp = ((df['output_is_buy'] == True) & (df['prediction_is_buy'] == True)).sum()
    tn = ((df['output_is_buy'] == False) & (df['prediction_is_buy'] == False)).sum()
    fp = ((df['output_is_buy'] == False) & (df['prediction_is_buy'] == True)).sum()
    fn = ((df['output_is_buy'] == True) & (df['prediction_is_buy'] == False)).sum()

    winning_rate = float(tp / (tp + fp)) if (tp + fp) > 0 else 0

    return {'tp': tp, 'tn': tn, 'fp': fp, 'fn': fn, 'winning_rate': winning_rate}

def get_profits(df):
    trimmed_average_profit = hf.get_trimmed_average(df['output_profit'], pct_to_trim=0.03, min_num_to_trim=8)
    average_profit = df['output_profit'].mean()
    median_profit = df['output_profit'].median()

    return {
        'trimmed_average_profit': trimmed_average_profit,
        'average_profit': average_profit,
        'median_profit': median_profit
    }

def get_loss_limit_pct(df):
    return df['output_is_loss_limit_reached'].sum() / len(df) if len(df) > 0 else 0

def get_performance_score(trimmed_average_profit, count, **kwargs):
    target_future_days = kwargs.get('target_future_days')

    target_count = 2 * test_size / (num_tickers * target_future_days)
    adjusted_profit = trimmed_average_profit ** 5 # to decrease small values, e.g. 0.8^2 = 0.64
    performance_score = adjusted_profit * min(count, target_count)
    
    return performance_score

def evaluate_model(df_data, model, test_train_data, hyperparams):
    df_test = slice_df_test(df_data, test_size)
    df_test = add_predictions(df_test, model, test_train_data['X_test'], **hyperparams)
    
    market_rate = get_market_rate(test_train_data['y_test'], **hyperparams)

    binary_classification = get_binary_classification(df_test)
    
    df_prediction_is_buy = df_test[(df_test['prediction_is_buy'] == True)]
    df_prediction_is_buy.to_excel(f'./outputs/{hf.get_date()}_classifier_df_prediction_is_buy.xlsx')

    profits = get_profits(df_prediction_is_buy)
    prediction_is_buy_count = len(df_prediction_is_buy['output_profit'])
    loss_limit_reached_pct = get_loss_limit_pct(df_prediction_is_buy)
    performance_score = get_performance_score(profits['trimmed_average_profit'],
                                              prediction_is_buy_count, **hyperparams)

    performance_metrics = {
        'performance_score': performance_score,
        **profits,
        'prediction_is_buy_count': prediction_is_buy_count,
        'loss_limit_reached_pct': loss_limit_reached_pct,
        'market_rate': market_rate,
        **binary_classification,
        'winning_rate_vs_market': binary_classification['winning_rate'] - market_rate,
    }

    return performance_metrics

In [9]:
from itertools import product

i = 0
results = []

def objective(hyperparams):
    df_data = get_df_data(hyperparams)
    test_train_data, model = load_model(df_data, hyperparams)
    performance_metrics = evaluate_model(df_data, model, test_train_data, hyperparams)

    result = {**performance_metrics, **hyperparams, 'epochs': epochs}
    results.append(result)

    performance = result['performance_score']

    return -performance

if use_hyperopt:
    best = fmin(objective, search_space, algo=tpe.suggest, max_evals=hyperopt_n_iterations)
    print(f'best parameters: {best}')
else:
    for params in product(*param_grid.values()):
        i += 1
        hf.print_combination(i, num_combinations)

        hyperparams = dict(zip(param_grid.keys(), params))

        df_data = get_df_data(hyperparams)
        test_train_data, model = load_model(df_data, hyperparams)
        performance_metrics = evaluate_model(df_data, model, test_train_data, hyperparams)

        result = {**performance_metrics, **hyperparams, 'epochs': epochs}
        print(result)
        results.append(result)


need to create ./outputs/classifier_transformed_data.pkl
saved new ./outputs/classifier_transformed_data.pkl    
need to create ./outputs/classifier_model.keras        
  0%|          | 0/600 [00:24<?, ?trial/s, best loss=?]

/Users/alexis/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 1/3                                              

   1/2179 ━━━━━━━━━━━━━━━━━━━━ 56:13 2s/step - accuracy: 0.1562 - loss: 6.8181
  33/2179 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2277 - loss: 4.6055  
  56/2179 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2342 - loss: 4.2918
  86/2179 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2342 - loss: 4.1203
 120/2179 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2348 - loss: 3.9817
 149/2179 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2362 - loss: 3.8965
 185/2179 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2371 - loss: 3.8275
 222/2179 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2384 - loss: 3.7672
 253/2179 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2394 - loss: 3.7229
 289/2179 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2405 - loss: 3.6747
 324/2179 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2417 - loss: 3.6324
 358/2179 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2425 - loss: 3.5946
 396/2179 ━━━━━

/Users/alexis/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



   1/8880 ━━━━━━━━━━━━━━━━━━━━ 3:34:27 1s/step - accuracy: 0.3750 - loss: 3.2990
  31/8880 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - accuracy: 0.2916 - loss: 3.0314   
  67/8880 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.2839 - loss: 2.9962
 117/8880 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.2853 - loss: 2.9187
 168/8880 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.2869 - loss: 2.8561
 218/8880 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.2886 - loss: 2.8102
 269/8880 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.2901 - loss: 2.7706
 320/8880 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.2912 - loss: 2.7379
 372/8880 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.2918 - loss: 2.7115
 423/8880 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.2921 - loss: 2.6889
 475/8880 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.2922 - loss: 2.6684 
 528/8880 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.2922 - loss: 2.6496
 580/8880 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.29

...

Epoch 1/3                                                                              

   1/2584 ━━━━━━━━━━━━━━━━━━━━ 56:56 1s/step - accuracy: 0.2188 - loss: 5.4325
  30/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2240 - loss: 4.9497  
  60/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2285 - loss: 4.6008
  90/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2315 - loss: 4.4403
 118/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2333 - loss: 4.3330
 140/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2354 - loss: 4.2706
 167/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2377 - loss: 4.2056
 194/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2400 - loss: 4.1520
 220/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2418 - loss: 4.1087
 247/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2434 - loss: 4.0682
 275/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2448 - loss: 4.0306
 302/2584 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.245

/Users/alexis/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



   1/1093 ━━━━━━━━━━━━━━━━━━━━ 25:26 1s/step - accuracy: 0.1953 - loss: 7.7734
  23/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1834 - loss: 6.6635  
  47/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1936 - loss: 6.4144
  71/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1986 - loss: 6.2636
  94/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2008 - loss: 6.1639
 114/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2024 - loss: 6.0897
 135/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2040 - loss: 6.0224
 154/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2054 - loss: 5.9689
 176/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2067 - loss: 5.9173
 190/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2073 - loss: 5.8881
 204/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2079 - loss: 5.8622
 226/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2085 - loss: 5.8263
 241/1093 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2089 - loss: 5.

/Users/alexis/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



  1/636 ━━━━━━━━━━━━━━━━━━━━ 12:45 1s/step - accuracy: 0.1094 - loss: 4.8214
 28/636 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1870 - loss: 4.8062  
 59/636 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1915 - loss: 4.6560
 90/636 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1968 - loss: 4.5251
104/636 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1990 - loss: 4.4806
129/636 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2020 - loss: 4.4168
157/636 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2048 - loss: 4.3604
184/636 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2070 - loss: 4.3127
211/636 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2087 - loss: 4.2739
239/636 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2103 - loss: 4.2393
267/636 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2118 - loss: 4.2085
294/636 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2131 - loss: 4.1828
322/636 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2142 - loss: 4.1577
350/636 ━━━━━━━━━━━━━

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by='performance_score', ascending=False)
df_results.head(1000)

In [ ]:
df_results.to_excel(f'./outputs/{hf.get_date()}_classifier_results.xlsx')